In [41]:
%load_ext autoreload
%autoreload 2
import dlib
import os
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torchvision
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:

def reverse_resized_rect(rect,resize_ratio):
    l = int(rect.left() / resize_ratio)
    t = int(rect.top() / resize_ratio)
    r = int(rect.right() / resize_ratio)
    b = int(rect.bottom() / resize_ratio)
    new_rect = dlib.rectangle(l,t,r,b)
    
    return [l,t,r,b] , new_rect

def resize_image(img, default_max_size=800):
    old_height, old_width, _ = img.shape
    if old_width > old_height:
        resize_ratio = default_max_size / old_width
        new_width, new_height = default_max_size, int(old_height * resize_ratio)
    else:
        resize_ratio = default_max_size / old_height
        new_width, new_height =  int(old_width * resize_ratio), default_max_size
    img = dlib.resize_image(img, cols=new_width, rows=new_height)
    return img, resize_ratio

def softmax_numpy(logits):
    exps = np.exp(logits - np.max(logits))  # Subtract max for numerical stability
    return exps / np.sum(exps)

In [51]:
os.chdir("/usr/users/vhassle/psych_track/FairFace")

#image_path = "test/child_test_1.jpg"
image_path = "/usr/users/vhassle/datasets/example_images/children/good_quality.png"

rects = []

#get image as wanted
img = dlib.load_rgb_image(image_path)
img, resize_ratio = resize_image(img)
print(img.shape)


##finds the faces in the image
cnn_face_detector = dlib.cnn_face_detection_model_v1('dlib_models/mmod_human_face_detector.dat')
sp = dlib.shape_predictor('dlib_models/shape_predictor_5_face_landmarks.dat')
dets = cnn_face_detector(img, 1) #takes the longest
num_faces = len(dets)
faces = dlib.full_object_detections()

for detection in dets:
    rect = detection.rect
    faces.append(sp(img, rect))
    rect_tpl ,rect_in_origin = reverse_resized_rect(rect,resize_ratio)
    rects.append(rect_in_origin)

# seems to extract the faces and size them to 300x300
images = dlib.get_face_chips(img, faces, size=300, padding = 0.25) #predefined
bboxes = rects

(464, 800, 3)


In [64]:
# now the prediction part
device = "cuda:0"
model_path = "fair_face_models/res34_fair_align_multi_7_20190809.pt"

trans = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                     std=[0.229, 0.224, 0.225])
])

model_fair_7 = torchvision.models.resnet34(weights=True)
model_fair_7.fc = nn.Linear(model_fair_7.fc.in_features, 18)
model_fair_7.load_state_dict(torch.load(model_path))
model_fair_7 = model_fair_7.to(device)
model_fair_7.eval()

# Now prediction of the images
#zoom on one image
observations = []
for i, image in enumerate(images[0:]):
    image = trans(image)
    image = image.view(1, 3, 224, 224) 
    image = image.to(device)
    outputs = model_fair_7(image)

    outputs = model_fair_7(image)
    outputs = outputs.cpu().detach().numpy()
    outputs = np.squeeze(outputs)
    # plt.imshow(images[i])
    # plt.show()

    ## Postprocessing 
    #race_outputs = outputs[:7]
    gender_outputs = outputs[7:9]
    age_outputs = outputs[9:18]

    #race_score = softmax_numpy(race_outputs)
    gender_score = softmax_numpy(gender_outputs)
    age_score = softmax_numpy(age_outputs)

    gender_pred = np.argmax(gender_score)
    age_pred = np.argmax(age_score)


    observations.append([gender_pred, age_pred,
                         gender_score, age_score])
observations


result = pd.DataFrame(observations)
result.columns = ['gender_preds',
                    'age_preds',
                    'gender_scores',
                    'age_scores',
                    ]

#bboxes

/usr/users/vhassle/miniforge3/envs/MOTIP/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [65]:
# Mapping for gender predictions
gender_mapping = {0: 'Male', 1: 'Female'}
age_mapping = {
    0: '0-2', 1: '3-9', 2: '10-19', 3: '20-29',
    4: '30-39', 5: '40-49', 6: '50-59', 7: '60-69', 8: '70+'
}

result['gender_preds'] = result['gender_preds'].map(gender_mapping)
result['age_preds'] = result['age_preds'].map(age_mapping)
result

,gender_preds,age_preds,gender_scores,age_scores
0,Female,3-9,"[0.014583065, 0.98541695]","[0.00044868793, 0.50427026, 0.43396738, 0.0239..."
1,Male,3-9,"[0.7906311, 0.20936885]","[0.0063302903, 0.95460147, 0.03841754, 0.00038..."
2,Female,3-9,"[0.4658541, 0.5341459]","[7.394971e-05, 0.7748002, 0.22416294, 0.000872..."
3,Male,3-9,"[0.69856405, 0.30143595]","[0.026177593, 0.56725216, 0.3014587, 0.0831028..."
